In [1]:
%pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import os
import numpy as np

# Excel file is loaded
file_path = 'JointAccountBudget.xlsx'
budgetDf = pd.read_excel(file_path, sheet_name='Joint Budget')

# Monthly statements are loaded
folder_path = 'monthlyStatementsCsvs'
monthlyStatementCsv = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Load and concatenate all CSVs into one DataFrame
df_combined = pd.concat(
    [pd.read_csv(os.path.join(folder_path, file)) for file in monthlyStatementCsv],
    ignore_index=True
)


In [3]:
# Keeping only relevant columns
df_combined = df_combined[['Date', 'Type', 'Name', 'Category', 'Amount', 'Notes and #tags']]

# Dropping what we put into the joint account - only care about expenses
df_combined = df_combined[~((df_combined['Amount'] > 0) | (df_combined['Type'].str.lower() == 'pot transfer'))]

#Converting negatives to positives
df_combined['Amount'] = df_combined['Amount'].abs()

In [4]:
# Categorising main expenses against existing budget categories
budgetCategoryMapping_names = {
    'Mortgage': ['Mark Botwood'],
    'Energy': ['ovo', 'fuse energy'],
    'Wifi': ['sky'],
    'Dates': ['pub', 'restaurant', 'cafe', 'bar', 'costa', 'starbucks','cinema', 'food','drink','dining','room'],
    'Service Charge': ['Harrington Green'],
    'Water': ['Severn Trent Water'],
    'Council Tax': ['Nottm City Council'],
    'Shopping': ['aldi', 'lidl', 'tesco', 'sainsbury', 'asda', 'McDonalds', 'Uber Eats', 'Just Eat', 'amazon'],
    'Netflix': ['netflix'],
    'Storage Container': ['quickselfs'],
    'Emergency Buffer': ['tbd'],
    'Honeymoon': ['tbd']
}

# If name maps to the keywords inside each category, assign that category as a new column. 
def assign_category_name(name):
    for category, keywords in budgetCategoryMapping_names.items():
        if any(keyword.lower() in name.lower() for keyword in keywords):
            return category
    return 'Unknown/Unmapped'  # Moved outside the loop
    
df_combined['Budget Category'] = df_combined['Name'].apply(assign_category_name)

# Truncate date to month and group everything by month and budget category - basically preparing for when I want to compare it against the budget set
df_combined['Date'] = pd.to_datetime(df_combined['Date'], format='%d/%m/%Y')
df_combined['Date'] = df_combined['Date'].dt.to_period('M').dt.to_timestamp()

groupedExpenses = df_combined.groupby(['Date', 'Budget Category'])['Amount'].sum().reset_index()


display(groupedExpenses)


,Date,Budget Category,Amount
0,2025-10-01,Council Tax,172.00
1,2025-10-01,Dates,37.55
2,2025-10-01,Energy,92.21
3,2025-10-01,Mortgage,525.00
4,2025-10-01,Netflix,5.99
5,2025-10-01,Service Charge,130.23
6,2025-10-01,Shopping,188.30
7,2025-10-01,Storage Container,59.34
8,2025-10-01,Unknown/Unmapped,215.82
9,2025-10-01,Water,57.90


In [5]:
groupedExpenses['Budget Category lower'] = groupedExpenses['Budget Category'].str.lower()
budgetDf['Expense lower'] = budgetDf['Expense'].str.lower()

expensesMapped = pd.merge(
    groupedExpenses,
    budgetDf[['Expense', 'Amount', 'Expense lower']],
    left_on='Budget Category lower',
    right_on='Expense lower',
    how='left'
)
expensesMapped.drop(columns=['Budget Category lower', 'Expense lower'], inplace=True)
expensesMapped.rename(columns={'Amount_x': 'Actual Amount', 'Amount_y': 'Budgeted Amount'}, inplace=True)
expensesMapped.fillna({
    'Budgeted Amount': 0,
    'Expense': 'Unknown/Unmapped'
}, inplace=True)
#expensesMapped['Expense'] = np.where(expensesMapped['Expense'] == 'NaN', 'Unknown/Unmapped', expensesMapped['Expense'])

# Actual budget check
expensesMapped['Within Budget?'] = expensesMapped['Actual Amount'] <= expensesMapped['Budgeted Amount']

In [7]:
monthAndYear = pd.Timestamp.now().strftime('%B_%Y')
expensesMapped.to_csv(f'budgetAnalysis_{monthAndYear}.csv', index=False)